In [1]:
# some code implemented from youtube tutorial from Madhav Thaker
# https://github.com/madhavthaker/spotify-recommendation-system

In [2]:
import pandas as pd
import numpy as np
import json
from IPython.display import Image
from IPython.core.display import display, HTML
import itertools
import sys
import re
import os
import matplotlib.pyplot as plt
from skimage import io
from IPython.display import Image


# Spotipy
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy.oauth2 import SpotifyOAuth
import spotipy.util as util

import warnings
warnings.filterwarnings("ignore")

%matplotlib inline
display(HTML("<style>.container { width:95% !important; }</style>"))
pd.set_option('display.max_columns', None)
pd.set_option("max_rows", None)

#prediction
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.svm import OneClassSVM
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors

url = os.getcwd()

## Extra

In [3]:
from secrets import client_id, client_secret

scope = 'user-library-read'

if len(sys.argv) > 1:
    username = sys.argv[1]
else:
    print("Usage: %s username" % (sys.argv[0],))
    sys.exit()

# Spotify access
auth_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(auth_manager=auth_manager)

token = util.prompt_for_user_token(scope, client_id= client_id, client_secret=client_secret, redirect_uri='http://localhost:8881/')
sp = spotipy.Spotify(auth=token)
spotify_username = sp.current_user()['id']

In [4]:
# display function that was show in a youtube tutorial from Madhav Thaker
def cover_songs(df):
    image_url = df['image_url'].values
    plt.figure(figsize=(15,int(0.625 * len(image_url))))
    columns = 5
    
    for i, url in enumerate(image_url):
        plt.subplot(len(image_url) / columns + 1, columns, i + 1)

        image = io.imread(url)
        plt.imshow(image)
        plt.xticks(color = 'w', fontsize = 0.1)
        plt.yticks(color = 'w', fontsize = 0.1)
        plt.xlabel(df['name'].values[i], fontsize = 12)
        plt.tight_layout(h_pad=0.4, w_pad=0)
        plt.subplots_adjust(wspace=None, hspace=None)

    plt.show()

## Gathering data

In [5]:
def get_users_playlists():
    playlist_names = {}
    playlist_photos = {}
    
    for i in sp.current_user_playlists()['items']:
        playlist_names[i['name']] = i['uri'].split(':')[2]
        playlist_photos[i['uri'].split(':')[2]] = i['images'][0]['url']
    
    return playlist_names, playlist_photos

# Call get_users_playlists()
playlist_names, playlist_photos = get_users_playlists()

# Here are all your playlist, copy the playlist id you want to choose songs from
playlist_names

{'New Playlist 2': '5e8ijEDzlaB0y9apSoZNBI',
 'Tyler Bates – The Punisher (Original Soundtrack) 2': '3qt3t2qfyWSli9F6C9C3SF',
 'Globus – Epicon (UK Version)': '25sJ1ieBzb2AVPS9GaZhKm',
 's': '2L0LTJqWOFFI8qPqkDVjKG',
 'Immediate – Trailerhead: Triumph': '71DD8rTMqJB4bnFRdS1Jz7',
 'Electric Romeo – Immediate': '2aL9qUB2A6vc72I4AwrWlt',
 'Highway man': '4thvxQYvACp32gZguRAiEw',
 'party mood': '7MIO3PrQ0ZXdPobIpHOc5a',
 'Southpaw (Music From And Inspired By The Motion Picture)': '0kF5WxQpwIlFG130bWovut',
 'Work Hard, Play Hard – Wiz Khalifa': '1CavHtskIP3Dp0r9RCwXpY',
 'From edits': '5Yi5I6uqMn9oyV3RhvUVNm',
 'beuty': '7K0flcn5YDOXxPRhChKzre',
 'OAZ': '0RVTSPELkBvNXUw27BUC0R',
 'Eyeex': '2zqTk2BwhJx4CaqhwZ40RK',
 'Icelandic Music | Ragnarsmedia': '2eyFd9OYGYRFG1WknFYfTy',
 'Íslenskt': '4jp0RPWEON6B3hbLu6f1N4',
 'Grime ': '3VsZqGKOP5E6pcixUKg8zt',
 'Ultra 1.0': '0fQrVALjgMV5ckiOG6Lw0u',
 'Ultra 2.0': '6N1CYumwsQ6hScprqxmfsL',
 'Ultra 3.0': '1thpoCGLWqyLpPMFli6dHD',
 '21': '3WCO6yNpxaBEtRVT

In [6]:
def get_playlist_tracks(username,playlist_id):
    results = sp.user_playlist_tracks(username,playlist_id)
    tracks = results['items']
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    return tracks

In [7]:
## Song Audio Features
sp.audio_features("17DjxJJG1NHXg1VatxfFfU")

[{'danceability': 0.693,
  'energy': 0.646,
  'key': 7,
  'loudness': -6.484,
  'mode': 1,
  'speechiness': 0.0313,
  'acousticness': 0.0602,
  'instrumentalness': 0.00132,
  'liveness': 0.678,
  'valence': 0.89,
  'tempo': 145.116,
  'type': 'audio_features',
  'id': '17DjxJJG1NHXg1VatxfFfU',
  'uri': 'spotify:track:17DjxJJG1NHXg1VatxfFfU',
  'track_href': 'https://api.spotify.com/v1/tracks/17DjxJJG1NHXg1VatxfFfU',
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/17DjxJJG1NHXg1VatxfFfU',
  'duration_ms': 222320,
  'time_signature': 4}]

In [8]:
## Song Audio Features
#sp.audio_features("17DjxJJG1NHXg1VatxfFfU")
def get_track_features(track_id):
    audio_features = sp.audio_features(track_id)[0]
    track_out = {
        'acousticness': audio_features['acousticness'],
        'danceability': audio_features['danceability'],
        'duration_ms': audio_features['duration_ms'],
        'energy': audio_features['energy'],
        'instrumentalness': audio_features['instrumentalness'],
        'liveness': audio_features['liveness'],
        'loudness': audio_features['loudness'],
        'mode': audio_features['mode'],
        'speechiness': audio_features['speechiness'],
        'tempo': audio_features['tempo'],
        'valence': audio_features['valence']
    }
    return track_out

In [9]:
def get_track_info(playlist_id, tracks, number):
    # Track
    track = tracks[number]
    added_by = track['added_by']['id']
    track_info = track['track']
    track_artist = track_info['artists'][0]
    track_album = track_info['album']

    # Informations about the track
    id = track_info['id']
    name = track_info['name']
    date_added = track['added_at']
    explicit = track_info['explicit']
    popularity = track_info['popularity']
    link = track_info['external_urls']['spotify']
    preview_url = track_info['preview_url']
    image_url = track_album['images'][1]['url']
    release_date = track_album["release_date"]
    
    # Informations about the artists
    artist_id = track_artist['id']
    artist_name = track_artist['name']
    
    ## audio features
    audio_features = sp.audio_features(id)[0]
    #print(audio_features)
    #track_features = get_track_features(id) 
    
    album_genres = sp.album(track_album['id'])
    #print("album genres:", album_genres)
    
    #'playlist_id': playlist_id,
    #'date_added': date_added,
    track_out = {
        'id': id,
        'name': name,
        'link': link,
        'popularity': popularity,
        'image_url': image_url,
        'preview_url': preview_url,
        'artist_id': artist_id,
        'artist_name': artist_name,
        'explicit': explicit,
        'release_date': release_date,
        'acousticness': audio_features['acousticness'],
        'danceability': audio_features['danceability'],
        'duration_ms': audio_features['duration_ms'],
        'energy': audio_features['energy'],
        'instrumentalness': audio_features['instrumentalness'],
        'liveness': audio_features['liveness'],
        'loudness': audio_features['loudness'],
        'mode': audio_features['mode'],
        'speechiness': audio_features['speechiness'],
        'tempo': audio_features['tempo'],
        'valence': audio_features['valence']
    }

    return track_out
 
#genre_seeds = sp.recommendation_genre_seeds()
#genres = genre_seeds['genres']

#test_track_info = specific_playlist_songs("5Yi5I6uqMn9oyV3RhvUVNm")
#test_track_info

In [10]:
def specific_playlist_songs(id_playlist):
    track_info = []
    for playlist_name, playlist_id in playlist_names.items():
        if(str(playlist_id) == id_playlist):
            tracks = get_playlist_tracks(spotify_username, playlist_id)
            for track_number in range(0, len(tracks)):
                track_info.append(get_track_info(playlist_id, tracks, track_number))
    return pd.DataFrame(track_info)

In [11]:
''' 
## I never finished implementing this function: 
## Maybe future work
def get_all_playlist_songs():
    track_info = []
    for playlist_name, playlist_id in playlist_names.items():
        tracks = get_playlist_tracks(spotify_username, playlist_id)
        for track_number in range(0, len(tracks)):
            track_info.append(get_track_info(playlist_id, tracks, track_number))
    return pd.DataFrame(track_info)    
'''

' \n## I never finished implementing this function: \n## Maybe future work\ndef get_all_playlist_songs():\n    track_info = []\n    for playlist_name, playlist_id in playlist_names.items():\n        tracks = get_playlist_tracks(spotify_username, playlist_id)\n        for track_number in range(0, len(tracks)):\n            track_info.append(get_track_info(playlist_id, tracks, track_number))\n    return pd.DataFrame(track_info)    \n'

In [12]:
## Creating playlist tracks
from_edits_playlist = specific_playlist_songs("5Yi5I6uqMn9oyV3RhvUVNm")
from_isx_playlist = specific_playlist_songs("4jp0RPWEON6B3hbLu6f1N4")

# Change here for your playlist id
#custom = specific_playlist_songs("0RVTSPELkBvNXUw27BUC0R")

In [ ]:
from_oaz = specific_playlist_songs("0RVTSPELkBvNXUw27BUC0R")

## Visualize the data

In [ ]:
## Uncomment to visually see the dataframe
#from_edits_playlist
#from_edits_playlist
#beuty
#custom

In [ ]:
cover_songs(from_edits_playlist)
#cover_songs(from_oaz)

In [ ]:
cover_songs(from_isx_playlist)

In [ ]:
## from_oaz.head()

## Predicting data

In [ ]:
def create_clean_copy(df):
    df["year"] = df['release_date'].apply(lambda x: x.split('-')[0])
    df = df.drop(columns = ['duration_ms', 'link', 'image_url', 'preview_url', 'artist_id', 'artist_name', 'release_date'])
    return df

In [ ]:
# change original_playlist = to the dataframe you want to use
original_playlist = from_oaz
clean_playlist = create_clean_copy(original_playlist)

In [ ]:
clean_playlist

In [ ]:
def display_outlier_data(clean_playlist, count, y_test):
    print("total songs in playlist: " + str(len(clean_playlist)))
    print("test outliers count: " + str(count))
    print("test outliers %:" +  str({count/len(y_test)*100}))
    outliers_df = pd.DataFrame(json.loads(outlier_temp_playlist)).T
    cover_songs(outliers_df)

In [ ]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import Normalizer
def predict_data_first(df):
    cleaned = df.drop(columns=['name', 'id'])
    X_train, X_test, y_train, y_test = train_test_split(cleaned, df['id'], test_size=0.30)
    
    ## SVM
    clf = make_pipeline(StandardScaler(), OneClassSVM(nu = 0.11, gamma=0.04))
    clf.fit(X_train, y_train)
    
    predictions = clf.predict(X_test)
    outliers = [id for id, predict in zip(y_test, predictions) if predict < 0]
    count = len(outliers)
    return predictions, outliers, count, y_test

In [ ]:
predictions, outliers, count, y_test = predict_data_first(clean_playlist)
outlier_temp_playlist = original_playlist.loc[clean_playlist['id'].isin(outliers)].to_json(orient='index')
display_outlier_data(from_oaz, count, y_test)

In [ ]:
detected_outliers = pd.DataFrame(json.loads(outlier_temp_playlist)).T
detected_outliers

## Extra

In [ ]:
## show all recommendations

In [ ]:
## show bad recommendations (low popularity)
#cover_songs(outliers_df)

In [ ]:
## show good recommendations (high popularity)